In [5]:
import mxnet as mx
from mxnet import image
import gluoncv
import matplotlib.pyplot as plt
from gluoncv.data.transforms.presets.segmentation import test_transform
from gluoncv.utils.viz import get_color_pallete,plot_image
import matplotlib.image as mpimg
import numpy as np
import math
from PIL import Image
from tqdm import tqdm
import os

# 源文件夹路径
source_folder = r"C:\Users\wac\Desktop\test0902"
# 目标文件夹路径
save_folder1 = r"C:\Users\wac\Desktop\test0902\test718_deeplab_resnest101_ade1"
save_folder2 = r"C:\Users\wac\Desktop\test0902\test718_deeplab_resnest101_ade2"
save_folder3 = r"C:\Users\wac\Desktop\test0902\test718_deeplab_resnest101_ade3"
save_folder4 = r"C:\Users\wac\Desktop\test0902\test718_deeplab_resnest101_ade4"

if not os.path.exists(save_folder1):
    os.mkdir(save_folder1)
if not os.path.exists(save_folder2):
    os.mkdir(save_folder2)
if not os.path.exists(save_folder3):
    os.mkdir(save_folder3)
if not os.path.exists(save_folder4):
    os.mkdir(save_folder4)

In [6]:
# using cpu
ctx = mx.cpu()
model_list = gluoncv.model_zoo.get_model_list()
model_list

dict_keys(['resnet18_v1', 'resnet34_v1', 'resnet50_v1', 'resnet101_v1', 'resnet152_v1', 'resnet18_v2', 'resnet34_v2', 'resnet50_v2', 'resnet101_v2', 'resnet152_v2', 'resnest14', 'resnest26', 'resnest50', 'resnest101', 'resnest200', 'resnest269', 'se_resnet18_v1', 'se_resnet34_v1', 'se_resnet50_v1', 'se_resnet101_v1', 'se_resnet152_v1', 'se_resnet18_v2', 'se_resnet34_v2', 'se_resnet50_v2', 'se_resnet101_v2', 'se_resnet152_v2', 'vgg11', 'vgg13', 'vgg16', 'vgg19', 'vgg11_bn', 'vgg13_bn', 'vgg16_bn', 'vgg19_bn', 'alexnet', 'densenet121', 'densenet161', 'densenet169', 'densenet201', 'shufflenet_v1', 'shufflenet_v2', 'squeezenet1.0', 'squeezenet1.1', 'googlenet', 'inceptionv3', 'xception', 'xception71', 'mobilenet1.0', 'mobilenet0.75', 'mobilenet0.5', 'mobilenet0.25', 'mobilenetv2_1.0', 'mobilenetv2_0.75', 'mobilenetv2_0.5', 'mobilenetv2_0.25', 'mobilenetv3_large', 'mobilenetv3_small', 'mobile_pose_resnet18_v1b', 'mobile_pose_resnet50_v1b', 'mobile_pose_mobilenet1.0', 'mobile_pose_mobilenetv

In [7]:
model1 = gluoncv.model_zoo.get_model('deeplab_resnest101_ade', pretrained=True)

In [3]:
model2 = gluoncv.model_zoo.get_model('deeplab_resnet101_citys', pretrained=True)

In [6]:
model3 = gluoncv.model_zoo.get_model('monodepth2_resnet18_posenet_kitti_mono_stereo_640x192', pretrained=True)

42433KB [00:02, 18569.77KB/s]                           


56017KB [00:03, 17833.34KB/s]                           


In [8]:
for imgName in tqdm(os.listdir(source_folder)):
    # 判断是否为照片
    if imgName.endswith(".jpg") or imgName.endswith(".png"):
        imgPath = os.path.join(source_folder,imgName)
        img = image.imread(imgPath)
        img = test_transform(img,ctx)
        output = model1.predict(img)
        predict = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()
        #print(predict)

        # 保存predict到save_folder中
        save_file_name = os.path.splitext(imgName)[0] + ".npy"
        save_path1 = os.path.join(save_folder1, save_file_name)
        np.save(save_path1, predict)

        # 保存图像1到save_folder1
        save_path2 = os.path.join(save_folder2,imgName)
        #mask = get_color_pallete(predict, 'citys')
        mask = get_color_pallete(predict, 'ade20k')
        mask = mask.convert('RGB')
        mask.save(save_path2, 'JPEG')
        
        # 保存图像2到save_folder2
        # 创建一个新的RGB图像，将预测中值为10的像素设为白色(255,255,255)，其他设为灰色(128,128,128)
        sky_image = Image.fromarray(np.zeros_like(predict, dtype=np.uint8), mode='L').convert('RGB')
        sky_image_array = np.array(sky_image)
        sky_image_array[(predict != 2)] = (128, 128, 128)
        # 将预测中值为2的像素设为白色(255,255,255)
        sky_image_array[predict == 2] = (255, 255, 255)
        # 更新图像
        sky_image = Image.fromarray(sky_image_array, mode='RGB')
        # 保存图像到目标文件夹
        save_path3 = os.path.join(save_folder3, imgName)
        sky_image.save(save_path3)
       
        # 保存图像3到save_folder3
                                                                                                                                                                                                                                                
        sl_image = Image.fromarray(np.zeros_like(predict, dtype=np.uint8), mode='L').convert('RGB')
        sl_image_array = np.array(sl_image)
        sl_image_array[(predict != 87)] = (128, 128, 128)
        sl_image_array[predict == 87] = (0, 0, 255)
        sl_image = Image.fromarray(sl_image_array, mode='RGB')
        save_path4 = os.path.join(save_folder4, imgName)
        sl_image.save(save_path4)

                   


100%|██████████| 5/5 [00:18<00:00,  3.63s/it]
